# Re-producing the Shift data
from the study "Flying in 2050"

## Hypothesis

In [2]:
import numpy as np
import configparser

%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use("./ipynb_inline.mplstyle")

from aviation import traj_objective, traj_ref, optimisation_traj

In [3]:
config = configparser.ConfigParser()
configfilename = "./aviation_parameters.ini"
config.read(configfilename)

['./aviation_parameters.ini']

In [4]:
begin_year = float(config["InitialState"]["begin_year"])
emission_2018_fr = float(config["InitialState"]["emission_2018_fr"])
emission_2018_wd = float(config["InitialState"]["emission_2018_wd"])
budget_carbone_2018_50_fr = float(config["InitialState"]["budget_carbone_2018_50_fr"])
budget_carbone_2018_50_wd = float(config["InitialState"]["budget_carbone_2018_50_wd"])
years = np.arange(begin_year, 2051, 1. )

In [5]:

trajectory_ref_word = traj_ref(config, years)
        
trajectory_obj_word = traj_objective(config, years)


In [6]:
from scipy.interpolate import make_interp_spline, BSpline

# 300 represents number of points to make between T.min and T.max
yearsnew = np.linspace(years.min(), years.max(), 300) 

optimisation_shortterm_operation = float(config["simpleModel"]["optimisation_shortterm_operation"])
optimisation_technologique = float(config["simpleModel"]["optimisation_technologique"])

spl = make_interp_spline(years, trajectory_ref_word, k=3, bc_type=([(1, 0.0)], [(2, 0.0)]))  # type: BSpline

traj_operation_wd = optimisation_traj(trajectory_ref_word, optimisation_shortterm_operation)
traj_techno_wd = optimisation_traj(traj_operation_wd, optimisation_technologique)

spl_op = make_interp_spline(years, traj_operation_wd, k=3, bc_type=([(1, 0.0)], [(2, 0.0)]))  # type: BSpline
spl_tech = make_interp_spline(years, traj_techno_wd, k=3, bc_type=([(1, 0.0)], [(2, 0.0)]))  # type: BSpline


# Making a Bokeh plot

In [51]:
from bokeh.io import output_notebook, output_file

from bokeh.layouts import gridplot, layout, Spacer
from bokeh.layouts import column, row
from bokeh.models import CustomJS, Slider
from bokeh.plotting import ColumnDataSource, figure, output_file, show, save

from bokeh.models import Span
output_notebook()

Loading BokehJS ...

In [270]:
x = yearsnew
ref_traj = spl(yearsnew)
op_traj = spl_op(yearsnew)
tech_traj = spl_tech(yearsnew)
obj_spl = make_interp_spline(years,trajectory_obj_word , k=3, bc_type=([(1, 0.0)], [(2, 0.0)]))  # type: BSpline

objectif_traj = obj_spl(yearsnew)

source = ColumnDataSource(data=dict(x=x, 
                                    ref_traj=ref_traj, 
                                    ref_traj_int=np.cumsum(ref_traj)/1000*(x[1]-x[0]), 
                                    oper_trj=op_traj, 
                                    oper_trj_int=np.cumsum(op_traj)/1000*(x[1]-x[0]), 
                                    tech_traj=tech_traj, 
                                    tech_traj_int=np.cumsum(tech_traj)/1000*(x[1]-x[0]), 
                                    objectif_traj=objectif_traj,
                                    objectif_traj_int=np.cumsum(objectif_traj)/1000*(x[1]-x[0]),
                                    ))


In [271]:


plot = figure( plot_width=850,
              plot_height=500,
              toolbar_location="below",
              toolbar_sticky=False,
              tools= "crosshair,pan,wheel_zoom,box_zoom,save,reset",
             background_fill_color="#fafafa")

# configure so that no drag tools are active
plot.toolbar.active_drag = None

plot.line('x', 'ref_traj',
          source=source,
          line_width=3,
          line_alpha=0.9,
          line_color="red", 
          legend_label="Reference +4%/yr")

plot.line('x', 'objectif_traj',
          source=source,
          line_width=3,
          line_alpha=0.6,
          line_color="black", 
          line_dash="dashed",
          legend_label="Objectif -3.39%/yr")

plot.line('x', 'oper_trj',
          source=source,
          line_width=3,
          line_alpha=0.6,
          legend_label="optimisation Operation")

plot.line('x', 'tech_traj',
          source=source,
          line_width=3,
          line_alpha=0.6,
          line_color="green",
          legend_label="optimisation Technologique")



plot.legend.location = "top_left"
plot.legend.click_policy="mute"

plot.legend.label_text_font_size = '8pt'

plot.title.text = 'Evolution of the World total emission per year'
plot.xaxis.axis_label = "year"
plot.yaxis.axis_label = "Emission rate [MtCO2/year]"

In [272]:
plot_integral = figure( plot_width=850,
              plot_height=500,
              toolbar_location="below",
              toolbar_sticky=False,
              tools= "crosshair,pan,wheel_zoom,box_zoom,save,reset",
             background_fill_color="#fafafa")

# configure so that no drag tools are active
plot_integral.toolbar.active_drag = None

plot_integral.line('x', 'ref_traj_int',
          source=source,
          line_width=3,
          line_alpha=0.9,
          line_color="red", 
          legend_label="Reference +4%/yr")

plot_integral.line('x', 'objectif_traj_int',
          source=source,
          line_width=3,
          line_alpha=0.6,
          line_color="black", 
          line_dash="dashed",
          legend_label="Objectif -3.39%/yr")

plot_integral.line('x', 'oper_trj_int',
          source=source,
          line_width=3,
          line_alpha=0.6,
          legend_label="optimisation Operation")

plot_integral.line('x', 'tech_traj_int',
          source=source,
          line_width=3,
          line_alpha=0.6,
          line_color="green",
          legend_label="optimisation Technologique")

vline = Span(location=20,
             dimension='width',
             line_color='black',
             line_width=1,
             )

plot_integral.add_layout(vline)
plot_integral.line([2019], [0],
                  legend_label="Objectif",
                  line_color='black',
                  line_width=1, line_alpha=1)

plot_integral.legend.location = "top_left"
plot_integral.legend.click_policy="mute"

plot_integral.legend.label_text_font_size = '8pt'

plot_integral.title.text = 'Evolution of the World total emission from 2019'
plot_integral.xaxis.axis_label = "year"
plot_integral.yaxis.axis_label = "Total Emission [GtCO2]"

In [279]:

opt_slider = Slider(start=0,
                    end=99,
                    value=50,
                    step=1,
                    title="Total Technological Optimisation in 2050 (%) ",
                    width=300, height=40)
op_slider = Slider(start=0,
                   end=30,
                   value=10,
                   step=1,
                   title="Total Operation Optimisation in 2050 (%) ",
                   width=300, height=40)

callback = CustomJS(args=dict(source=source,
                              opt_op=op_slider,
                              opt_tech=opt_slider,
                             ),
                    code="""
    const data = source.data;
    const tech_percent = opt_tech.value;
    const op_perc = opt_op.value;

    const x = data['x']
    const tech_traj = data['tech_traj']
    const ref_traj = data['ref_traj']
    const oper_trj = data['oper_trj']
    
    const factor_op = 1-(1-op_perc/100) ** (1/ (x.length) )
    const factor_tech = 1-(1-tech_percent/100) ** (1/ (x.length) )
    
    for (var i = 0; i < x.length; i++) {
        oper_trj[i] = ref_traj[i] * (1 - factor_op) ** (i) ;
    }
    
    for (var i = 0; i < x.length; i++) {
        tech_traj[i] = oper_trj[i] * (1 - factor_tech) ** (i) ;
    }
    
    const oper_trj_int = data['oper_trj_int']
    const tech_traj_int = data['tech_traj_int']

    oper_trj.reduce(function(a,b,i) { return oper_trj_int[i] = (a+b/1000*(x[1]-x[0])); },0);
    tech_traj.reduce(function(a,b,i) { return tech_traj_int[i] = (a+b/1000*(x[1]-x[0])); },0);
    
    source.change.emit();
""")

opt_slider.js_on_change('value', callback)
op_slider.js_on_change('value', callback)





layout = column( [ row( [ opt_slider, op_slider], height=50, width=800, sizing_mode="fixed" ), 
                  plot,
                  plot_integral
              ], sizing_mode='stretch_both', max_height=900, min_height=400, max_width=750
)


#output_file("transport_interactive_evolution.html", title="transport_evolution")



In [280]:
show(layout)

In [281]:
from bokeh.resources import CDN
from bokeh.plotting import figure
from bokeh.embed import autoload_static

folderpath = "/home/tavant/Documents/blogs/myblog/static/"
scriptfilename = "/js/shift_aviation_2050.js"
divfilename = "/js/shift_aviation_2050.html"


js, tag = autoload_static(layout, CDN, script_path=scriptfilename)

with open(folderpath+scriptfilename, "w") as f:
    f.write(js)
    
print(tag)

with open(folderpath+divfilename, "w") as fp:
    fp.write(tag)
      


<script src="/js/shift_aviation_2050.js" id="3e2fc19b-2839-4045-9499-278e80e33d60"></script>


# Bio Carb

In [282]:
perf_techno_year = 0.02
traj_performances_wd = optimisation_traj(trajectory_ref_word, factor_yearly=perf_techno_year)
spl_perf = make_interp_spline(years, traj_performances_wd, k=3, bc_type=([(1, 0.0)], [(2, 0.0)]))  # type: BSpline
perfo_traj = spl_perf(yearsnew)


In [283]:
flot_altcarb_usability = np.ones_like(years)*0.5
flot_newmodels = np.zeros_like(years)
altCarb_disponibility = np.zeros_like(years)

index2035 = np.argwhere(years==2035)[0][0]
indexMan = len(years)

renewDuration = 20 # years
renewRate = 1/renewDuration

for i in range(index2035+1, indexMan):
    flot_newmodels[i] = flot_newmodels[i-1] + renewRate

flot_newmodels[flot_newmodels>1] = 1

for i in range(index2035, indexMan):
    flot_altcarb_usability[i] = flot_newmodels[i] * 1.0 + (1-flot_newmodels[i]) * 0.5

altCarb_disponibility[0] = 1.001

for i in range(1, indexMan):
    altCarb_disponibility[i] = altCarb_disponibility[i-1]**1.2

altCarb_disponibility -= altCarb_disponibility[0]
altCarb_disponibility /= altCarb_disponibility[-1]
altCarb_disponibility *= 500   # 500MT total capacuity of Fuil, Times 3 for the CO2 masse

carbdispo_spl = make_interp_spline(years,altCarb_disponibility , k=3, bc_type=([(1, 0.0)], [(2, 0.0)]))  # type: BSpline
carbdispo_traj = carbdispo_spl(yearsnew)


In [284]:
traj_altCarb_wd  = traj_performances_wd.copy()

for i in range(len(years)):
    traj_altCarb_wd[i] -=  altCarb_disponibility[i]* 2.85
traj_altCarb_wd[-1]

trajcarb_spl = make_interp_spline(years,traj_altCarb_wd , k=3, bc_type=([(1, 0.0)], [(2, 0.0)]))  # type: BSpline
trajcarb = trajcarb_spl(yearsnew)

In [301]:


source = ColumnDataSource(data=dict(x=x,
                                    ref_traj_int=np.cumsum(ref_traj)/1000*(x[1]-x[0]),
                                    ref_traj=ref_traj,
                                    perfo_traj=perfo_traj,
                                    perfo_traj_int=np.cumsum(perfo_traj)/1000*(x[1]-x[0]),
                                    objectif_traj=objectif_traj,
                                    objectif_traj_int=np.cumsum(objectif_traj)/1000*(x[1]-x[0]),
                                   carbdispo_traj=carbdispo_traj,
                                   carbdispo_traj_ref=carbdispo_traj,
                                   trajcarb=trajcarb,
                                   trajcarb_int=np.cumsum(trajcarb)/1000*(x[1]-x[0]),
                                   ))

plot = figure( plot_width=850,
              plot_height=500,
              #toolbar_location="below",
              toolbar_sticky=False,
              tools= "crosshair,pan,wheel_zoom,box_zoom,save,reset",
             background_fill_color="#fafafa")

plotCarb = figure( plot_width=850,
              plot_height=300,
              #toolbar_location="below",
              toolbar_sticky=False,
              tools= "crosshair,pan,wheel_zoom,box_zoom,save,reset",
             background_fill_color="#fafafa")

# configure so that no drag tools are active
plot.toolbar.active_drag = None
plotCarb.toolbar.active_drag = None

plotCarb.line('x', 'carbdispo_traj',
          source=source,
          line_width=3,
          line_alpha=0.9,
          line_color="red",
          legend_label="Senario production")

plotCarb.line('x', 'carbdispo_traj_ref',
          source=source,
          line_width=3,
          line_alpha=0.9,
          line_color="blue",
          legend_label="Reference production")
plotCarb.title.text ="Production of Bio fuel and PTC (Mt)"

plotCarb.legend.location = "top_left"
plotCarb.legend.click_policy="mute"
plotCarb.legend.label_text_font_size = '8pt'

plot.line('x', 'ref_traj',
          source=source,
          line_width=3,
          line_alpha=0.9,
          line_color="red", 
          legend_label="Reference +4%/yr")

plot.line('x', 'objectif_traj',
          source=source,
          line_width=3,
          line_alpha=0.6,
          line_color="black", 
          line_dash="dashed",
          legend_label="Objectif -3.39%/yr")

plot.line('x', 'perfo_traj',
          source=source,
          line_width=3,
          line_alpha=0.6,
          legend_label="Performance")

plot.line('x', 'trajcarb',
          source=source,
          line_width=3,
          line_alpha=0.6,
          line_color="green",
          legend_label="Using Alt. fuel")

plot.legend.location = "top_left"
plot.legend.click_policy="mute"

plot.legend.label_text_font_size = '8pt'

plot.title.text = 'Evolution of the World total emission per year'
plot.xaxis.axis_label = "year"
plot.yaxis.axis_label = "Emission [MtCO2/year]"


In [302]:


plot_integral = figure( plot_width=850,
              plot_height=500,
              # toolbar_location="below",
              toolbar_sticky=False,
              tools= "crosshair,pan,wheel_zoom,box_zoom,save,reset",
             background_fill_color="#fafafa")

# configure so that no drag tools are active
plot_integral.toolbar.active_drag = None

plot_integral.line('x', 'objectif_traj_int',
          source=source,
          line_width=3,
          line_alpha=0.6,
          line_color="black", 
          line_dash="dashed",
          legend_label="Objectif -3.39%/yr")

plot_integral.line('x', 'perfo_traj_int',
          source=source,
          line_width=3,
          line_alpha=0.6,
          legend_label="Performance")

plot_integral.line('x', 'trajcarb_int',
          source=source,
          line_width=3,
          line_alpha=0.6,
          line_color="green",
          legend_label="Using Alt. fuel")


vline = Span(location=20,
             dimension='width',
             line_color='black',
             line_width=1,
             )

#plot_integral.add_layout(vline)
#plot_integral.line([2019], [0],
#                  legend_label="Objectif",
#                  line_color='black',
#                  line_width=1, line_alpha=1)


plot_integral.legend.location = "top_left"
plot_integral.legend.click_policy="mute"

plot_integral.legend.label_text_font_size = '8pt'

plot_integral.title.text = 'Evolution of the World total emission'
plot_integral.xaxis.axis_label = "year"
plot_integral.yaxis.axis_label = "Emission [GtCO2]"


In [305]:

opt_slider = Slider(start=0, end=5, value=2, step=.1,
                    title="Perf. improvement (%/year) ", width=180, height=40)
maxcarb_slider = Slider(start=0, end=900, value=500, step=10,
                   title="Alt. fuel produced (Mt) ", width=180, height=40)
maxcarb_slider_factore = Slider(start=0, end=100, value=100, step=1,
                   title="Part of Alt. fuel used (%) ", width=180, height=40)


callback = CustomJS(args=dict(source=source,
                              maxcarb=maxcarb_slider,
                              opt_tech=opt_slider,
                              maxcarb_part=maxcarb_slider_factore,
                             ),
                    code="""
    const data = source.data;
    const tech_percent = opt_tech.value;
    const maxcarb_value = maxcarb.value;
    const maxcarb_part_value = maxcarb_part.value;

    const x = data['x']
    const perfo_traj = data['perfo_traj']
    const ref_traj = data['ref_traj']
    const carbdispo_traj = data['carbdispo_traj']
    const carbdispo_traj_ref = data['carbdispo_traj_ref']
    const trajcarb = data['trajcarb']
    
    const total_tech =  ( 1 - tech_percent/100 ) ** (2050-2019)
    const factor_tech = 1 - ( total_tech) ** (1/ (x.length) )
    
    for (var i = 0; i < x.length; i++) {
        perfo_traj[i] = ref_traj[i] * (1 - factor_tech) ** (i) ;
    }
    
    for (var i = 0; i < x.length; i++) {
        carbdispo_traj[i] = carbdispo_traj_ref[i] * maxcarb_value / carbdispo_traj_ref[x.length -1];
    }
    
    for (var i = 0; i < x.length; i++) {
        trajcarb[i] = perfo_traj[i] -  carbdispo_traj[i] * 2.85 * maxcarb_part_value/100 ;
        trajcarb[i] = Math.max(trajcarb[i], 0)
    }
    
    const perfo_traj_int = data['perfo_traj_int']
    const trajcarb_int = data['trajcarb_int']

    perfo_traj.reduce(function(a,b,i) { return perfo_traj_int[i] = (a+b/1000*(x[1]-x[0])); },0);
    trajcarb.reduce(function(a,b,i) { return trajcarb_int[i] = (a+b/1000*(x[1]-x[0])); },0);


    source.change.emit();
""")

opt_slider.js_on_change('value', callback)
maxcarb_slider.js_on_change('value', callback)
maxcarb_slider_factore.js_on_change('value', callback)





#layout = column( [ row( [ column([opt_slider, maxcarb_slider, maxcarb_slider_factore ], width=200, height=200),
#                       Spacer(width=100, height=200)],
#                         height=150, width=300, sizing_mode="fixed" ), 
#                  plotCarb,
#                  plot,
#                  plot_integral,
#              ], sizing_mode='stretch_both',
#                max_height=1200,
#                min_height=800,
#                max_width=750
#)


layout = row([ column([Spacer(width=230, height=300), 
                      maxcarb_slider,Spacer(width=230, height=60),
                      maxcarb_slider_factore, Spacer(width=230, height=60),
                      opt_slider, ], width=190, height=350), 
             column( [ plotCarb, plot, plot_integral], sizing_mode='stretch_both',
                max_height=1200,
                min_height=800,
                max_width=750),]
                ,height=900, width=750, sizing_mode="fixed" )

#output_file("transport_interactive_evolution.html", title="transport_evolution")



In [306]:
show(layout)

In [307]:
scriptfilename = "/js/shift_aviation_2050_secteur.js"
divfilename = "/js/shift_aviation_2050_secteur.html"


js, tag = autoload_static(layout, CDN, script_path=scriptfilename)

with open(folderpath+scriptfilename, "w") as f:
    f.write(js)
    
print(tag)

with open(folderpath+divfilename, "w") as fp:
    fp.write(tag)
      


<script src="/js/shift_aviation_2050_secteur.js" id="d932d511-ada9-42fa-9aa5-ea1093926e42"></script>
